# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [4]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3


In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_prompt = """You are a helpful assistant. Your Goal is help answer user questions in regards to our company's policies , users may ask about refunds , shipping , returns , and other policies."""

In [11]:
DB = "policies2.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS policies (question TEXT PRIMARY KEY, answer REAL)')
    conn.commit()

In [12]:
def set_policy(question, answer):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        
        # 1. Use 'question' and 'answer' from your function arguments
        # 2. Fix 'questions' -> 'question'
        # 3. Use 'excluded.answer' to update the value if it already exists
        sql = '''
            INSERT INTO policies (question, answer) 
            VALUES (?, ?) 
            ON CONFLICT(question) 
            DO UPDATE SET answer = excluded.answer
        '''
        
        cursor.execute(sql, (question.lower(), answer))
        conn.commit()

In [13]:
set_policy("what is the return policy" , "30 days with reciept")
set_policy("what is the return policy for more than 30 days" , "no return")
set_policy("when is my item going to arrive" , "2 days")

In [56]:
def get_policy(question):
    print(f"DATABASE TOOL CALLED: Searching for {question}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        
        # Use LIKE and % for partial matching
        # This will find "return policy" inside "What is the return policy?"
        search_query = f"%{question.lower()}%"
        cursor.execute('SELECT answer FROM policies WHERE question LIKE ?', (search_query,))
        
        result = cursor.fetchone()
        
        if result:
            return result[0]
        return "I am sorry, I couldn't find a specific policy for that in my database."

In [57]:
get_policy('what is the return policy')

DATABASE TOOL CALLED: Searching for what is the return policy


'30 days with reciept'

In [58]:
get_policy_function = {
    "name": "get_policy",
    "description": "Get the policy for a given question.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to look up in the policy database",
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [59]:
tools = [{"type": "function", "function": get_policy_function}]

In [60]:
tools[0]['function']

{'name': 'get_policy',
 'description': 'Get the policy for a given question.',
 'parameters': {'type': 'object',
  'properties': {'question': {'type': 'string',
    'description': 'The question to look up in the policy database'}},
  'required': ['question'],
  'additionalProperties': False}}

In [61]:
def handle_tool_calls(message):
    tool_call = message.tool_calls[0]

    if tool_call.function.name == "get_policy":
        arguments = json.loads(tool_call.function.arguments)

        user_question = arguments.get('question')

        policy_data = get_policy(user_question)

        response = [{
            "role" : "tool",
            "content" : policy_data,
            "tool_call_id" : tool_call.id
        }]
        
    return response

In [62]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses_list = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses_list)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [63]:
gr.ChatInterface(fn=chat, title="Chat with me", description="Chat with me" , type='messages').launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Searching for return policy
DATABASE TOOL CALLED: Searching for shipping time
DATABASE TOOL CALLED: Searching for dog delivery time
